In [77]:
import pandas as pd
import numpy as np
from EvGym import config
import cvxpy as cp
import sys


In [78]:
x = np.arange(10_000) + 50
y = np.random.rand(10_000)

df_tbl = pd.DataFrame({"x": x, "y": y})
df_tbl.head()

,x,y
0,50,0.560872
1,51,0.991841
2,52,0.531892
3,53,0.501677
4,54,0.893476


In [79]:
df_ibl = df_tbl.copy()
df_ibl = df_ibl.set_index("x")
df_ibl.head()

,y
x,
50,0.560872
51,0.991841
52,0.531892
53,0.501677
54,0.893476


In [80]:
for i in x:
    row = df_tbl[df_tbl["x"] == i].iloc[0]


In [81]:
for i in x:
    row = df_ibl.loc[i]

In [82]:
df_price = pd.read_csv("./data/df_price_2019.csv")
df_price = df_price.set_index(["ts"])
df_price.head(30)

,date,price_da,price_im
ts,,,
166560.0,2019-01-01 00:00:00,0,0.04801
166561.0,2019-01-01 01:00:00,0,0.03903
166562.0,2019-01-01 02:00:00,0,0.02539
166563.0,2019-01-01 03:00:00,0,0.03095
166564.0,2019-01-01 04:00:00,0,0.03885
166565.0,2019-01-01 05:00:00,0,0.02220
166566.0,2019-01-01 06:00:00,0,0.01985
166567.0,2019-01-01 07:00:00,0,0.03532
166568.0,2019-01-01 08:00:00,0,0.03626


In [83]:
idx_min = df_price.index.min().astype(int)
idx_max = df_price.index.max().astype(int)

for i in range(idx_min, idx_max + 1):
    try:
        row = df_price.loc[i]
    except KeyError:
        print(f"KeyError: {i}")

In [84]:
df_price.loc[166789]

date        2019-01-10 13:00:00
price_da                      0
price_im                0.04367
Name: 166789.0, dtype: object

In [85]:
df_sessions = pd.read_csv(f"{config.data_path}/df_elaad_preproc.csv", parse_dates = ["starttime_parking", "endtime_parking"])
df_sessions[["ts_arr"]].head()


,ts_arr
0,166560.0
1,166570.0
2,166571.0
3,166572.0
4,166572.0


In [86]:
df_sessions["ts_arr"].min()

166560.0

In [87]:
print(df_sessions["ts_dep"].max())
print(df_sessions["ts_arr"].max())

175317.0
175313.0


In [88]:
ts_dep_max = df_sessions["ts_dep"].max()
df_max_dep = df_sessions[df_sessions["ts_dep"] == ts_dep_max]
df_max_dep

,session,ChargePoint,Connector,starttime_parking,endtime_parking,StartCard,connected_time_float,charged_time_float,total_energy,max_power,...,soc_arr,charged_time,connected_time,ts_arr,ts_dep,ts_soj,laxity,depart_hour,xi,laxity_float
9995,3633983,853f3c2b6f0637daeed95f8c2fdd5dab,2,2019-12-31 16:49:00,2019-12-31 21:04:00,197601e126b5336c668d3384253ae88b9eadec7aac01d6...,4.25,3.48,38.81,18.0,...,0.50107,4.0,5.0,175312.0,175317.0,5.0,1.52,21.0,8.191128,0.77


In [89]:
# KeyError: 168699.0

In [90]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 8760 entries, 166560.0 to 175319.0
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      8760 non-null   object 
 1   price_da  8760 non-null   int64  
 2   price_im  8760 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 593.8+ KB


In [91]:
df_price = df_price.reset_index()
df_price[(168690 < df_price["ts"]) & (df_price["ts"] < 168710)].head(30)

,ts,date,price_da,price_im
2131,168691.0,2019-03-30 19:00:00,0,0.04790
2132,168692.0,2019-03-30 20:00:00,0,0.08622
2133,168693.0,2019-03-30 21:00:00,0,0.03134
2134,168694.0,2019-03-30 22:00:00,0,0.03385
2135,168695.0,2019-03-30 23:00:00,0,0.03190
2136,168696.0,2019-03-31 00:00:00,0,-0.03248
2137,168697.0,2019-03-31 01:00:00,0,0.09661
2138,168698.0,2019-03-31 02:00:00,0,0.03376
2139,168699.0,2019-03-31 03:00:00,0,0.02548
2140,168700.0,2019-03-31 04:00:00,0,0.02548


In [92]:

df = pd.DataFrame(data = [[1,2,3],[4,5,6],[7,8,9]], columns = ['a','b','c'])
display(df.head())
df.loc[df["a"] < 5, "c"] = df.loc[df["a"] < 5]["c"] + 1
display(df.head())

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


,a,b,c
0,1,2,4
1,4,5,7
2,7,8,9


In [93]:
all(df.loc[df["a"] < 5]["c"] > 3)


True

In [94]:
l_t0 = df.index[df["a"]==4].to_list()
if len(l_t0) == 1:
    t0 = l_t0[0]
    ls = df.iloc[t0:t0+2]["c"].values
    print(ls)
    print(type(ls))

[7 9]
<class 'numpy.ndarray'>


In [95]:
df.index.max()

2

In [96]:
df.iloc[0:2]

,a,b,c
0,1,2,4
1,4,5,7


In [97]:
i2 = df["a"] == 7
print(len(i2))
print(type(i2))
print(i2)
print(i2.sum())

3
<class 'pandas.core.series.Series'>
0    False
1    False
2     True
Name: a, dtype: bool
1


In [98]:
filt = df["a"] > 1
for idx, row in df[filt].iterrows():
    print(type(row))
    print(type(row.a))
    print(row["a"], row.b, row.c)

print()
for row in df[filt].itertuples():
    print(type(row))
    print(type(row.a))
    print(row.a, row.b, row.c)

<class 'pandas.core.series.Series'>
<class 'numpy.int64'>
4 5 7
<class 'pandas.core.series.Series'>
<class 'numpy.int64'>
7 8 9

<class 'pandas.core.frame.Pandas'>
<class 'int'>
4 5 7
<class 'pandas.core.frame.Pandas'>
<class 'int'>
7 8 9


In [99]:
df

,a,b,c
0,1,2,4
1,4,5,7
2,7,8,9


In [100]:
df_big = df[df["a"] > 1]
df.loc[df_big.index].apply(lambda row: row["c"] if row["b"] == 5 else 0, axis = 1).sum()

7

In [101]:
A = np.zeros((3,3))
A[2,1:] = 1
A

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 1., 1.]])

In [102]:
Y = cp.Variable((5,3))
price = np.array([2,3,4])
Y * price

constraints = []
objective_vec = []

constraints += [(Y * price) == 100 ]

/home/javier/.local/lib/python3.8/site-packages/cvxpy/interface/numpy_interface/ndarray_interface.py:47: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if result.dtype in [numpy.complex, numpy.float64]:
/home/javier/.local/lib/python3.8/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, U

In [103]:
A = np.array([[1.1,2.1,3.1],[1.2,2.2,3.2],[1.3,2.3,3.3]])
print(A)
x = np.array([10,3,2])
print(x)
print(A@x)
print(A*x)

[[1.1 2.1 3.1]
 [1.2 2.2 3.2]
 [1.3 2.3 3.3]]
[10  3  2]
[23.5 25.  26.5]
[[11.   6.3  6.2]
 [12.   6.6  6.4]
 [13.   6.9  6.6]]


In [104]:
A.sum()

19.8

In [105]:
np.asmatrix(x).T

/home/javier/.local/lib/python3.8/site-packages/numpy/matrixlib/defmatrix.py:69: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


matrix([[10],
        [ 3],
        [ 2]])

In [106]:
a = 1/9
print(f"{a:0.2f}")



0.11


In [108]:
sys.getsizeof(a)

24

In [110]:
# array2string can decrease the size of a np array
A_str = np.array2string(A, precision = 2, separator = ",")
print(f"{A=}, size {sys.getsizeof(A)}")
print(f"{A_str=}, size {sys.getsizeof(A_str)}")

A=array([[1.1, 2.1, 3.1],
       [1.2, 2.2, 3.2],
       [1.3, 2.3, 3.3]]), size 192
A_str='[[1.1,2.1,3.1],\n [1.2,2.2,3.2],\n [1.3,2.3,3.3]]', size 96
